<a href="https://colab.research.google.com/github/MatteoOnger/VIA_Project/blob/main/VIA_EQM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VIA Project: EqMotion Meets Assembly101 - Equivariant Motion Modeling for Manipulation Tasks**

*   **Author:** Matteo Onger
*   **Date:** September 2025

**Documentation**:
*   Dataset: [Assembly101](https://assembly-101.github.io/)
*   Model: [EqMotion](https://github.com/MediaBrain-SJTU/EqMotion)
*   Training Framework: [4DHands](https://github.com/FrancescoAgnelli3/4D_hands)

**Notes**:
*   For faster execution, please use a GPU-equipped runtime.

## Clone the GitHub repositories

In [1]:
# Check PyTorch and CUDA version
import torch
print(torch.__version__)

2.8.0+cu126


In [ ]:
# Install dependencies
!pip install torchcde
!pip install torch-dct
!pip install torch-geometric

!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.8.0+cu126.html       # Set PyTorch and CUDA version
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.8.0+cu126.html        # Set PyTorch and CUDA version

!pip install git+https://github.com/TorchSpatiotemporal/tsl.git

In [1]:
# Clone training framework
!git clone https://github.com/FrancescoAgnelli3/4D_hands -b feat-eqmotion

Cloning into '4D_hands'...
remote: Enumerating objects: 1012, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1012 (delta 3), reused 3 (delta 3), pack-reused 999 (from 2)
Receiving objects: 100% (1012/1012), 45.90 MiB | 33.64 MiB/s, done.
Resolving deltas: 100% (408/408), done.


In [2]:
# Clone original EqMotion repository
!git clone https://github.com/MediaBrain-SJTU/EqMotion.git

Cloning into 'EqMotion'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 210 (delta 46), reused 27 (delta 9), pack-reused 128 (from 1)
Receiving objects: 100% (210/210), 530.02 MiB | 54.48 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Updating files: 100% (162/162), done.


## Load the dataset

### From SFTP server

In [ ]:
from google.colab import userdata

In [ ]:
# Set the credentials and paths
local_path = '/content'
data_folder = 'data_our'
remote_path = '/datasets/AssemblyHands/assembly101-download-scripts/data_our'

host = userdata.get('phuse_server')     # Server domain must be SET in SECRETS
user = userdata.get('phuse_user')       # Server user must be SET in SECRETS
psw = userdata.get('phuse_psw')         # User password must be SET in SECRETS

user_at_host = user + '@' + host
data_path = local_path + '/' + data_folder

In [ ]:
# Install sshpass
!sudo apt update
!sudo apt install sshpass

# Create directories
!mkdir -p $remote_path
!mkdir -p ~/.ssh

# Add server to known hosts
!ssh-keyscan $host >> ~/.ssh/known_hosts

# Download the data
!sshpass -p $psw sftp -oBatchMode=no -b - $user_at_host <<< "get -r {remote_path} {local_path}"

### From drive

In [3]:
from google.colab import drive

In [4]:
# Mount Google Drive
drive.mount('/content/drive')

# Set the paths
data_path = '/content/drive/MyDrive/Colab Notebooks/UNIMI/VIA Project/data_our'

Mounted at /content/drive


### Mount the dataset

In [5]:
import os

In [6]:
# Mount the dataset
expected_path = '/content/4D_hands/Assembly/data'

if not os.path.exists(expected_path):
    os.symlink(data_path, expected_path)
    print(f'Symlink created: {expected_path} -> {data_path}')
else:
    print(f'Symlink or folder already exists: {expected_path}')

Symlink created: /content/4D_hands/Assembly/data -> /content/drive/MyDrive/Colab Notebooks/UNIMI/VIA Project/data_our


## Run the models

In [7]:
# Set working directory
%cd /content/4D_hands/Assembly/spatiotemporal

/content/4D_hands/Assembly/spatiotemporal


In [15]:
# Run the models
!python /content/4D_hands/Assembly/spatiotemporal/test_prediction.py \
    --dataset Assembly \
    --models eqmotion \
    --action rotate \
    --batch_size 128  \
    --time 100 \
    --window 90

Reshaping all the vectors to  100 time steps
Loading files: 100% 262/262 [00:36<00:00,  7.14it/s]
{Train dataloader: size=163}
{Validation dataloader: size=0}
{Test dataloader: size=40}
{Predict dataloader: None}
Number of model (eqmotion) parameters:      73668
Epoch [1/50] Train Loss: 0.401680 | Test Loss: 0.291223 | LR: 0.001000
Epoch [2/50] Train Loss: 0.328334 | Test Loss: 0.268260 | LR: 0.001000
Epoch [3/50] Train Loss: 0.316568 | Test Loss: 0.260787 | LR: 0.001000
Epoch [4/50] Train Loss: 0.310398 | Test Loss: 0.239261 | LR: 0.001000
Epoch [5/50] Train Loss: 0.296097 | Test Loss: 0.236181 | LR: 0.001000
Epoch [6/50] Train Loss: 0.293760 | Test Loss: 0.231825 | LR: 0.001000
Epoch [7/50] Train Loss: 0.282041 | Test Loss: 0.221148 | LR: 0.001000
Epoch [8/50] Train Loss: 0.273180 | Test Loss: 0.212319 | LR: 0.001000
Epoch [9/50] Train Loss: 0.263050 | Test Loss: 0.208881 | LR: 0.001000
Epoch [10/50] Train Loss: 0.262588 | Test Loss: 0.204931 | LR: 0.001000
Epoch [11/50] Train Loss: 

## Run original EqMotion model

In [16]:
#  set working directory
%cd /content/EqMotion

/content/EqMotion


In [19]:
# Run the model
!python /content/EqMotion/main_nbody.py --epochs 10

Namespace(exp_name='exp_1', batch_size=100, epochs=10, past_length=20, future_length=20, no_cuda=False, seed=-1, test_interval=5, outf='n_body_system/logs', lr=0.0005, nf=64, n_layers=4, max_training_samples=5000, dataset='nbody', weight_decay=1e-12, norm_diff=False, tanh=False, vis=False, apply_decay=False, cuda=True)
The seed is : 691
train epoch 0 avg loss: 2.34225
==> val epoch 0 avg loss: 1.35803 ade: 0.80328
==> test epoch 0 avg loss: 1.33868 ade: 0.80272
*** Best Val Loss: 1.35803 	 Best Test Loss: 1.33868 	 Best ade: 0.80272 	 Best epoch 0
train epoch 1 avg loss: 0.64751
train epoch 2 avg loss: 0.55971
train epoch 3 avg loss: 0.52482
train epoch 4 avg loss: 0.43304
train epoch 5 avg loss: 0.37496
==> val epoch 5 avg loss: 0.63357 ade: 0.33901
==> test epoch 5 avg loss: 0.62748 ade: 0.33815
*** Best Val Loss: 0.63357 	 Best Test Loss: 0.62748 	 Best ade: 0.33815 	 Best epoch 5
train epoch 6 avg loss: 0.32453
train epoch 7 avg loss: 0.31635
train epoch 8 avg loss: 0.30047
train e